In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

In [2]:
path = '../home-credit-default-risk/'
df = pd.read_csv(path + 'application_train.csv')

In [6]:
def prepare_data(df, drop=False):
    df_temp = df.copy()
    
    if drop:
        missing = df_temp.isna().mean()
        drop_cols = missing[missing > 0.5].index
        df_temp = df_temp.drop(columns=drop_cols)
    
    y = df_temp['TARGET']
    X = df_temp.drop('TARGET', axis=1)
    
    X = pd.get_dummies(X, dummy_na=True)
    
    return X, y

In [7]:
def train_and_eval(X, y, model_type='logistic', balance_weights=False):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    weight = 'balanced' if balance_weights else None

    steps = [
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()) 
    ]


    if model_type == 'logistic':
        steps.append(('model', LogisticRegression(
            class_weight=weight, 
            max_iter=1000,
            random_state=42
        )))
    elif model_type == 'random_forest':
        steps.append(('model', RandomForestClassifier(
            n_estimators=100,
            max_depth=10,
            class_weight=weight,
            random_state=42,
            n_jobs=-1
        )))
    elif model_type == 'gradient_boosting':
        if weight:
            print('There is no weight balance in a GD model.')
        steps.append(('model', GradientBoostingClassifier(
            n_estimators=100,
            max_depth=5,
            learning_rate=0.1,
            random_state=42
        )))
    else:
        raise ValueError(f"Unknown model type: {model_type}")

    pipeline = Pipeline(steps)


    pipeline.fit(X_train, y_train)

    preds = pipeline.predict_proba(X_test)[:, 1]
    score = roc_auc_score(y_test, preds)
    
    return score

Logistic regression

Without droping any columns

In [12]:
X1, y1 = prepare_data(df, drop=False)
score_1 = train_and_eval(X1, y1)
print(f"ROC AUC: {score_1:.4f}")

ROC AUC: 0.7485


In [16]:
X1, y1 = prepare_data(df, drop=False)
score_1 = train_and_eval(X1, y1, balance_weights=True)
print(f"ROC AUC: {score_1:.4f}")

ROC AUC: 0.7489


With droping columns that are >50% null

In [13]:
X2, y2 = prepare_data(df, drop=True)
score_2 = train_and_eval(X2, y2)
print(f"ROC AUC: {score_2:.4f}")

ROC AUC: 0.7451


In [17]:
X2, y2 = prepare_data(df, drop=True)
score_2 = train_and_eval(X2, y2, balance_weights=True)
print(f"ROC AUC: {score_2:.4f}")

ROC AUC: 0.7457


Ensemble methods

In [14]:
score_3 = train_and_eval(X1, y1, 'random_forest')
print(f"ROC AUC: {score_3:.4f}")

ROC AUC: 0.7363


In [18]:
score_3 = train_and_eval(X1, y1, 'random_forest', balance_weights=True)
print(f"ROC AUC: {score_3:.4f}")

ROC AUC: 0.7329


In [15]:
score_4 = train_and_eval(X1, y1, 'gradient_boosting')
print(f"ROC AUC: {score_4:.4f}")

ROC AUC: 0.7581


We see that baseline model - logistic regression even without any feature engineering and without any feature selection is getting roc auc of 0.74.

Random forest achieved worse score of 0.73.

Gradient boosting achieved the best score of almost 0.76.

On kaggle competition the best score is 0.806.

# with bureau 

In [4]:
from src.data_loader import merge_data
df = merge_data('../home-credit-default-risk')

df shape: (307511, 358)


In [5]:
df.shape

(307511, 358)

In [8]:
X1, y1 = prepare_data(df, drop=False)

In [10]:
score = train_and_eval(X1, y1)
print(f"ROC AUC: {score:.4f}")

ROC AUC: 0.7547


In [11]:
score = train_and_eval(X1, y1, 'random_forest')
print(f"ROC AUC: {score:.4f}")

ROC AUC: 0.7373


In [12]:
score = train_and_eval(X1, y1, 'gradient_boosting')
print(f"ROC AUC: {score:.4f}")

ROC AUC: 0.7627
